In [ ]:
import pandas as pd
import json, os
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" #CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8

import torch
import torch.nn as nn
import torch_geometric
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch.nn import Linear, LSTM, RNN, GRU, ReLU, Tanh, Sigmoid, CrossEntropyLoss

from torch.optim import Adam
from torch.nn.functional import cross_entropy
from torch.utils.data import random_split

from torch_geometric.data import Data, Batch
from torch_geometric.explain import Explainer, GNNExplainer, CaptumExplainer, PGExplainer, AttentionExplainer
from torch_geometric.nn import global_mean_pool, BatchNorm, global_max_pool, global_add_pool, TopKPooling, SAGPooling

from captum.attr import Saliency, IntegratedGradients

import numpy as np
import sklearn
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score
from sklearn.model_selection import KFold, StratifiedKFold

from IPython.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = 'cpu'
print('Device:', device)

if device == 'cuda':
    #torch.use_deterministic_algorithms(True)
    torch.backends.cudnn.benchmark = False
#     os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8" #CUBLAS_WORKSPACE_CONFIG=:4096:8 or CUBLAS_WORKSPACE_CONFIG=:16:8

In [ ]:
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)

In [ ]:
### Read Stored Graphs ###
# # Only Vocal Features
# graphs_females = torch.load('GNN_data_structure_females_v4_vocal.pt')
# graphs_males = torch.load('GNN_data_structure_males_v4_vocal.pt')
# # Only Facial Features
# graphs_females = torch.load('GNN_data_structure_females_v4_facial.pt')
# graphs_males = torch.load('GNN_data_structure_males_v4_facial.pt')
# # Only Verbal Features
# graphs_females = torch.load('GNN_data_structure_females_v4_verbal.pt')
# graphs_males = torch.load('GNN_data_structure_males_v4_verbal.pt')
# All Features
graphs_females = torch.load('GNN_data_structure_females_v4.pt')
graphs_males = torch.load('GNN_data_structure_males_v4.pt')

num_features = graphs_females[0].x.shape[-1]

### Move to GPU ###
batch_females = Batch.from_data_list(graphs_females).to(device)
batch_males = Batch.from_data_list(graphs_males).to(device)

print('Females:', len(graphs_females))
print('Males:', len(graphs_males))
print('Total:', len(graphs_females)+len(graphs_males))
print(len(batch_females.x))
print(len(batch_males.x))

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
        self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Females ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [150]
}
best_f1 = 0.0
best_params = None

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_females.cpu(), batch_females.y):
                train_batch = Batch.from_data_list([graphs_females[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_females[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                pred2 = eval_model(model, test_batch)
                
                acc_list.append(accuracy_score(y_true=test_batch.y.tolist(), y_pred=pred))
                prec_list.append(precision_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                recall_list.append(recall_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                f1_list.append(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred2, average='macro'))
            
            print(f1_list)
            # print(f'    Females Accuracy score: {sum(all_acc_list)/len(all_acc_list):.4f}')
            print(hidden_channels, epochs)
            print(f'    Females Precision: {np.mean(prec_list):.2f} ± {np.std(prec_list):.2f}')
            print(f'    Females Recall   : {np.mean(recall_list):.2f} ± {np.std(recall_list):.2f}')
            print(f'    Females F1       : {np.mean(f1_list):.2f} ± {np.std(f1_list):.2f}')
            print('-'*50)

            # Check if the current hyperparameters are the best so far
            f1 = sum(f1_list)/len(f1_list)
            if f1 > best_f1:
                best_f1 = f1
                best_params = {"hidden_channels": hidden_channels, "epochs": epochs}
# print('='*100)
# print(f"Best F1 score: {best_f1:.4f}")
# print(f"Best hyperparameters: {best_params}")

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
        self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.08)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Males ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [240]
}
best_f1 = 0.0
best_params = None

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_males.cpu(), batch_males.y):
                train_batch = Batch.from_data_list([graphs_males[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_males[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                pred2 = eval_model(model, test_batch)
                
                acc_list.append(accuracy_score(y_true=test_batch.y.tolist(), y_pred=pred))
                prec_list.append(precision_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                recall_list.append(recall_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                f1_list.append(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred2, average='macro'))
            
            print(f1_list)
            # print(f'    Females Accuracy score: {sum(all_acc_list)/len(all_acc_list):.4f}')
            print(hidden_channels, epochs)
            print(f'    Males Precision: {np.mean(prec_list):.2f} ± {np.std(prec_list):.2f}')
            print(f'    Males Recall   : {np.mean(recall_list):.2f} ± {np.std(recall_list):.2f}')
            print(f'    Males F1       : {np.mean(f1_list):.2f} ± {np.std(f1_list):.2f}')
            print('-'*50)

            # Check if the current hyperparameters are the best so far
            f1 = sum(f1_list)/len(f1_list)
            if f1 > best_f1:
                best_f1 = f1
                best_params = {"hidden_channels": hidden_channels, "epochs": epochs}
# print('='*100)
# print(f"Best F1 score: {best_f1:.4f}")
# print(f"Best hyperparameters: {best_params}")

In [ ]:
### Use Different Layers of GCN to compare ###

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.conv3 = GCNConv(int(hidden_channels/2), int(hidden_channels/4))
        self.conv4 = GCNConv(int(hidden_channels/4), num_classes)
#         self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
#         self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
#         x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Females ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [150]
}
best_f1 = 0.0
best_params = None

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_females.cpu(), batch_females.y):
                train_batch = Batch.from_data_list([graphs_females[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_females[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                pred2 = eval_model(model, test_batch)
                
                acc_list.append(accuracy_score(y_true=test_batch.y.tolist(), y_pred=pred))
                prec_list.append(precision_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                recall_list.append(recall_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                f1_list.append(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred2, average='macro'))
            
            print(f1_list)
            # print(f'    Females Accuracy score: {sum(all_acc_list)/len(all_acc_list):.4f}')
            print(hidden_channels, epochs)
            print(f'    Females Precision: {np.mean(prec_list):.2f} ± {np.std(prec_list):.2f}')
            print(f'    Females Recall   : {np.mean(recall_list):.2f} ± {np.std(recall_list):.2f}')
            print(f'    Females F1       : {np.mean(f1_list):.2f} ± {np.std(f1_list):.2f}')
            print('-'*50)

            # Check if the current hyperparameters are the best so far
            f1 = sum(f1_list)/len(f1_list)
            if f1 > best_f1:
                best_f1 = f1
                best_params = {"hidden_channels": hidden_channels, "epochs": epochs}
# print('='*100)
# print(f"Best F1 score: {best_f1:.4f}")
# print(f"Best hyperparameters: {best_params}")

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.conv3 = GCNConv(int(hidden_channels/2), int(hidden_channels/4))
        self.conv4 = GCNConv(int(hidden_channels/4), num_classes)
#         self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
#         self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.conv3(x, edge_index)
        x = self.conv4(x, edge_index)
#         x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.08)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Males ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [240]
}
best_f1 = 0.0
best_params = None

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_males.cpu(), batch_males.y):
                train_batch = Batch.from_data_list([graphs_males[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_males[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                pred2 = eval_model(model, test_batch)
                
                acc_list.append(accuracy_score(y_true=test_batch.y.tolist(), y_pred=pred))
                prec_list.append(precision_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                recall_list.append(recall_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                f1_list.append(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred, average='macro'))
                #print(f1_score(y_true=test_batch.y.tolist(), y_pred=pred2, average='macro'))
            
            print(f1_list)
            # print(f'    Females Accuracy score: {sum(all_acc_list)/len(all_acc_list):.4f}')
            print(hidden_channels, epochs)
            print(f'    Males Precision: {np.mean(prec_list):.2f} ± {np.std(prec_list):.2f}')
            print(f'    Males Recall   : {np.mean(recall_list):.2f} ± {np.std(recall_list):.2f}')
            print(f'    Males F1       : {np.mean(f1_list):.2f} ± {np.std(f1_list):.2f}')
            print('-'*50)

            # Check if the current hyperparameters are the best so far
            f1 = sum(f1_list)/len(f1_list)
            if f1 > best_f1:
                best_f1 = f1
                best_params = {"hidden_channels": hidden_channels, "epochs": epochs}
# print('='*100)
# print(f"Best F1 score: {best_f1:.4f}")
# print(f"Best hyperparameters: {best_params}")

In [ ]:
### Export Prediction of model and test for gender bias ###

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
        self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Females ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [150]
}
print("prediction,isFemale")

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_females.cpu(), batch_females.y):
                train_batch = Batch.from_data_list([graphs_females[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_females[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                
                for i in range(0, len(pred)):
                    print(str(pred[i]) + "," + "1")

In [ ]:
### Define GNN model ###
from torch_geometric.nn import GatedGraphConv, GATConv, SuperGATConv, MLP, GraphSAGE
from torch.optim import SGD
import torch.nn.init as init

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, int(hidden_channels/2))
        self.linear = Linear(int(hidden_channels/2), int(hidden_channels/4))
        self.lstm = LSTM(int(hidden_channels/4), num_classes)
    def forward(self, data):
        x, edge_index, batch = data.x, data.edge_index, data.batch
        # 1. Obtain node embeddings: Embed each node by performing multiple rounds of message passing
        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        x = self.linear(x)
        # 2. Readout layer: Aggregate node embeddings into a unified graph embedding (readout layer)
        x = global_mean_pool(x, batch) #, TopKPooling, SAGPooling# global_mean_pool(x, batch)
        # 3. Apply a final classifier: Train a final classifier on the graph embedding
        x = F.dropout(x, p=0.5, training=self.training)
        return F.log_softmax(x, dim=1)

def train_model(num_features, hidden_channels, epochs, train_batch):
    # Create an instance of the model
    model = GCN(num_features, hidden_channels, 2).to(device)
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=0.08)
    criterion = torch.nn.CrossEntropyLoss()

    # Train Model
    for epoch in range(epochs):
        optimizer.zero_grad()
        out = model(train_batch)
        loss = criterion(out, train_batch.y)
        loss.backward()
        optimizer.step()
    return model

def eval_model(model, test_batch):
    # Evaluate Model
    model.eval()
    out = model(test_batch)
    pred = out.argmax(dim=1).tolist()  # Use the class with highest probability.
    return pred

### Train and Evaluate Model (Stratified K-Fold) ###
print('Train and Evaluate Model (Stratified K-Fold)...')
torch.manual_seed(11)
torch_geometric.seed_everything(11)
np.random.seed(11)
num_folds = 5 # Define the number of folds

######################################################
### Males ###
######################################################
params = {
    "hidden_channels": [128],
    "epochs": [240]
}
best_f1 = 0.0
best_params = None
print("prediction,isFemale")

for seed in range(0, 1):
    # Create a KFold object
    skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=seed)
    
    # Iterate over all combinations of hyperparameters
    for hidden_channels in params["hidden_channels"]:
        for epochs in params["epochs"]:
            acc_list, prec_list, recall_list, f1_list = [], [], [], []

            # Use the KFold object to split the data into train and test sets
            for train_index, test_index in skf.split(batch_males.cpu(), batch_males.y):
                train_batch = Batch.from_data_list([graphs_males[i] for i in train_index]).to(device)
                test_batch = Batch.from_data_list([graphs_males[i] for i in test_index]).to(device)
                
                model = train_model(num_features, hidden_channels, epochs, train_batch)
                pred = eval_model(model, test_batch)
                for i in range(0, len(pred)):
                    print(str(pred[i]) + "," + "0")